## Strategia

In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
import numpy as np
import pandas as pd


#wyceniam opcje za pomoca Black Scholesa
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

In [2]:
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S*bs_call(S,K,T,r,sigma)

In [3]:
df = pd.read_csv("cocoa.csv")
df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

In [4]:
sigma = np.sqrt(252) * df['returns'].std() 

In [5]:
lcp = df['Close'].iloc[-1] #ostatnia cena, 
t = 90/365 #data wygasniecia , 3 m-ce
strike_price = 2400
r = 0.06 #r wolna od ryzyka

In [6]:
prem = bs_call(lcp, strike_price, t, 0.05, sigma) #robimy long calla (chyba)
prem

162.85687550812054

In [7]:
cena_o = prem + strike_price
cena_o

2562.8568755081205

In [8]:
ceny = np.arange(-500,500,1) + lcp #hipotetyczne ceny, zmiennosc co 1 dolar
#ceny

In [9]:
cena_opcji = []
for i in ceny:
    if i >= cena_o:
        cena_opcji.append(cena_o)
    else:
        cena_opcji.append(i+prem)

In [10]:
cena_opcji[:10]

[2104.8568755081205,
 2105.8568755081205,
 2106.8568755081205,
 2107.8568755081205,
 2108.8568755081205,
 2109.8568755081205,
 2110.8568755081205,
 2111.8568755081205,
 2112.8568755081205,
 2113.8568755081205]

In [11]:
d = {'Cena kakao':ceny,'cena opcji':cena_opcji}
porownanie = pd.DataFrame(d)
porownanie.iloc[495:505]

,Cena kakao,cena opcji
495,2437.0,2599.856876
496,2438.0,2600.856876
497,2439.0,2601.856876
498,2440.0,2602.856876
499,2441.0,2603.856876
500,2442.0,2604.856876
501,2443.0,2605.856876
502,2444.0,2606.856876
503,2445.0,2607.856876
504,2446.0,2608.856876


In [12]:
porownanie["zysk/strata"] = porownanie["Cena kakao"]-porownanie["cena opcji"]

In [17]:
porownanie.iloc[495:505]

,Cena kakao,cena opcji,zysk/strata,zysk/strata w procencie
495,2437.0,2599.856876,-162.856876,-0.066827
496,2438.0,2600.856876,-162.856876,-0.066799
497,2439.0,2601.856876,-162.856876,-0.066772
498,2440.0,2602.856876,-162.856876,-0.066745
499,2441.0,2603.856876,-162.856876,-0.066717
500,2442.0,2604.856876,-162.856876,-0.066690
501,2443.0,2605.856876,-162.856876,-0.066663
502,2444.0,2606.856876,-162.856876,-0.066635
503,2445.0,2607.856876,-162.856876,-0.066608
504,2446.0,2608.856876,-162.856876,-0.066581


In [14]:
porownanie["zysk/strata w procencie"] = porownanie["zysk/strata"]/porownanie["Cena kakao"]#nie wiem czy tak trzeba
porownanie

,Cena kakao,cena opcji,zysk/strata,zysk/strata w procencie
0,1942.0,2104.856876,-162.856876,-0.083860
1,1943.0,2105.856876,-162.856876,-0.083817
2,1944.0,2106.856876,-162.856876,-0.083774
3,1945.0,2107.856876,-162.856876,-0.083731
4,1946.0,2108.856876,-162.856876,-0.083688
...,...,...,...,...
995,2937.0,2562.856876,374.143124,0.127390
996,2938.0,2562.856876,375.143124,0.127687
997,2939.0,2562.856876,376.143124,0.127983
998,2940.0,2562.856876,377.143124,0.128280


## Obliczanie wartości zagrożonej

### Meoda historyczna zwykła

In [19]:
np.quantile(porownanie["zysk/strata w procencie"], 0.95)

0.11352038530344935

In [20]:
np.quantile(porownanie["zysk/strata w procencie"], 0.99)

0.12560623182433647